In [1]:
%autosave 60
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
import sklearn

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

import pickle
import os
import csv
import pandas as pd
import pickle


Autosaving every 60 seconds
Automatically created module for IPython interactive environment


In [2]:
FEATURE_TYPE = 'deepmag' #['raw', 'deepmag']
DATASET_FOR_SEVERITY_RATING = "tas2_severity_dataset.pkl"

if FEATURE_TYPE == 'raw':
    DATA_DIR = "C:/Wasif/PDMotorFeatureExtraction/TASK2_FEATURES_04_21/"
    X_file = "x_repeat_removed_raw_pixels.npy"
    y_file = "y_repeat_removed_raw_pixels.npy"
    X_index_file = "index_repeat_removed.pickle"

elif FEATURE_TYPE == 'deepmag':
    INPUT_DIR = "E:/Wasif/PDMotorFeatureExtraction/"
    assert(os.path.exists(INPUT_DIR))
    
    survey_file = "survey_april 9th 2021.csv"
    feature_file = "DeepMAGFeatures/deepMAG_features.pickle"
    superpd_survey_file = "SUPERPD_DATA_2020-03-24_1110.csv"
    
    DATA_DIR = "E:/Wasif/PDMotorFeatureExtraction/Task2_features_deepMAG/"
    X_file = "x_repeat_removed_deepmag.npy"
    y_file = "y_repeat_removed_deepmag.npy"
    X_index_file = "index_repeat_removed.pickle"
    
    rating_file = "task2_dr_saloni_rating.csv"
    
IMG_HEIGHT = 492
IMG_WIDTH = 492

In [3]:
#Rating CSV file from Dr. Saloni.
def read_rating_data():
    rating_full_path = os.path.join(INPUT_DIR,rating_file)
    data = pd.read_csv(rating_full_path)
    return data

#PARK users' self reported survey: PD/Non-PD, diagnosis time, medication
def read_survey(list_columns):
    survey_full_path = os.path.join(INPUT_DIR,survey_file)
    data = pd.read_csv(survey_full_path)
    data = data[list_columns]
    return data

#SuperPARK users' profile, obtained from clinic. 
def read_superpd_survey(list_columns):
    survey_full_path = os.path.join(INPUT_DIR,superpd_survey_file)
    data = pd.read_csv(survey_full_path)
    data = data[list_columns]
    return data

#Read extracted features (DeepMAG frequency features)
def read_features():
    pickle_filename = os.path.join(INPUT_DIR,feature_file)
    with open(pickle_filename, 'rb') as handle:
        features = pickle.load(handle)
    return features

## Process SuperPD Survey

In [4]:
def mins_to_hour(row):
    mins = row["last-medication"]
    if np.isnan(mins):
        return mins
    else:
        return mins/60.0
    
def num_to_yesno(row):
    diagnosed = row["diagnosed"]
    if np.isnan(diagnosed):
        return diagnosed
    else:
        if diagnosed==1.0:
            return "yes"
        else:
            return "no"

def concat_for_unique_id(row):
    patient_id = str(row['guid'])
    visit_date = str(row['visitdate'])
    return patient_id+"-task2-"+visit_date

#Read SuperPARK users clinical record
superpd_data = read_superpd_survey(['guid', 'visitdate', 'redcap_event_name', 
                                    'cohort', 'mdsupdrsptntprknsnmedind_2', 'mdsupdrsptclinstateprknsnm_2', 
                                    'mdsupdrsptntuseldopaind_2', 'mdsupdrslstldopadosetm_2'])
superpd_data = superpd_data.rename(columns={"mdsupdrsptntprknsnmedind_2": "on_pd_medication", "mdsupdrsptclinstateprknsnm_2": "on_off_state", "mdsupdrsptntuseldopaind_2": "on_levodopa", "mdsupdrslstldopadosetm_2":"last-medication"})
# guid = unique user id; cohort = PD status; 
# redcap_event_name = screening (first visit), baseline (second visit), month6_visit (third visit)
# mdsupdrsptntprknsnmedind_2 --> Is the patient on medication for treating the symptoms of Parkinson's Disease?
# mdsupdrsptclinstateprknsnm_2 --> If the patient is receiving medication for treating the symptoms of Parkinson's Disease, mark the patient's clinical state using the following definitions:
#0, ON (On is the typical functional state when patients are receiving medication and have a good response)
#1, OFF (Off is the typical functional state when patients have a poor response in spite of taking medications.)
# mdsupdrsptntuseldopaind_2 --> Is the patient on Levodopa?
# mdsupdrslstldopadosetm_2 --> If the patient is on Levodopa, minutes since last levodopa dose:
#cohort is the diagnosis result (1: PD/0: Non-PD).

superpd_data["last-medication"] = pd.to_numeric(superpd_data["last-medication"], errors='coerce')
superpd_data["on_off_state"] = pd.to_numeric(superpd_data["on_off_state"], errors='coerce')
#In super-pd survey, the unit is minutes, in normal PD survey, it is in hour
superpd_data["last-medication"] = superpd_data.apply(mins_to_hour, axis=1)

#Some users' cohort is N/A during baseline visit, but has a valid status in screening visit
#if the user with cohort N/A is assigned PD/Non-PD in any visit, assign PD/Non-PD to all instances of visits
superpd_data['cohort']=superpd_data['cohort'].fillna(-1.0)
superpd_data = superpd_data.join(superpd_data.groupby('guid')['cohort'].max(), on='guid', rsuffix='_corrected')
#One user is not assigned a status. Just ignore.
#superpd_data[superpd_data['cohort_corrected']==-1]
superpd_data = superpd_data.rename(columns={"cohort_corrected": "diagnosed"})

#Convert to yes/no to match normal PARK survey data
superpd_data["diagnosed"] = superpd_data.apply(num_to_yesno, axis=1)

#Make an id similar to normal PD survey
superpd_data["id"] = superpd_data.apply(concat_for_unique_id, axis=1)

superpd_data = superpd_data.drop(columns = ['cohort', 'on_pd_medication', 'on_levodopa', 'redcap_event_name', 'guid', 'visitdate'])

#Count instances where participants are in 'on' state and under the effect of medication
#superpd_data[(superpd_data["on_off_state"]==0) & (superpd_data["diagnosed"]=='yes') & ((superpd_data['last-medication']>=0.75) & (superpd_data['last-medication']<=3.0))].count()
superpd_data

,on_off_state,last-medication,diagnosed,id
0,NaN,NaN,yes,NIHAF261TBMPV-task2-2019-09-17
1,0.0,0.750000,yes,NIHAF261TBMPV-task2-2019-10-08
2,NaN,NaN,no,NIHAG749MLBRQ-task2-2020-01-13
3,NaN,NaN,no,NIHAG749MLBRQ-task2-2020-02-05
4,NaN,NaN,yes,NIHAV871KZCVE-task2-2019-08-02
...,...,...,...,...
157,NaN,NaN,no,NIHYW557MLDFE-task2-2020-03-20
158,NaN,NaN,no,NIHZT156UUPLX-task2-2020-01-21
159,NaN,NaN,no,NIHZT156UUPLX-task2-2020-02-14
160,NaN,NaN,yes,NIHZY217YWJA8-task2-2020-02-18


### Findings

1. 38 instances where participants are in 'on' state and under the effect of medication

## Process Normal Users Survey

In [5]:
#Read survey CSV of normal PARK users. 
#'repeat' means the user has already performed the test before. We are removing such cases.
survey_data = read_survey(['id','diagnosed','repeat', 'year-of-diagnosis', 'last-medication'])
survey_data = survey_data[survey_data["repeat"]!='yes']
#'repeat' column is no longer needed
survey_data = survey_data.drop(columns = ["repeat"])
#Make the ID consistent with feature_extraction pipeline
survey_data["id"] = survey_data["id"] + "-task2"
survey_data["year-of-diagnosis"] = pd.to_numeric(survey_data["year-of-diagnosis"], errors='coerce')
survey_data["last-medication"] = pd.to_numeric(survey_data["last-medication"], errors='coerce')
survey_data

,id,diagnosed,year-of-diagnosis,last-medication
0,2018-10-24T19-20-17-709Z8-task2,no,NaN,NaN
1,2018-09-19T22-50-04-770Z13-task2,yes,2015.0,4.0
2,2018-10-30T20-09-29-976Z87-task2,no,NaN,NaN
3,2018-09-11T16-58-31-717Z92-task2,yes,2016.0,NaN
4,2018-11-01T20-59-19-606Z70-task2,no,NaN,NaN
...,...,...,...,...
1083,2017-10-13T05-38-01-106Z41-task2,yes,2010.0,NaN
1084,2020-09-03T18-01-57-790Z80-task2,no,NaN,NaN
1085,2020-02-23T22-29-06-220Z56-task2,no,NaN,NaN
1086,2018-09-18T01-16-29-987Z51-task2,yes,2008.0,4.0


## Combine SuperPARK and PARK survey

In [6]:
combined_survey_data = pd.concat([survey_data, superpd_data], join='outer')
combined_survey_data

,id,diagnosed,year-of-diagnosis,last-medication,on_off_state
0,2018-10-24T19-20-17-709Z8-task2,no,NaN,NaN,NaN
1,2018-09-19T22-50-04-770Z13-task2,yes,2015.0,4.000000,NaN
2,2018-10-30T20-09-29-976Z87-task2,no,NaN,NaN,NaN
3,2018-09-11T16-58-31-717Z92-task2,yes,2016.0,NaN,NaN
4,2018-11-01T20-59-19-606Z70-task2,no,NaN,NaN,NaN
...,...,...,...,...,...
157,NIHYW557MLDFE-task2-2020-03-20,no,NaN,NaN,NaN
158,NIHZT156UUPLX-task2-2020-01-21,no,NaN,NaN,NaN
159,NIHZT156UUPLX-task2-2020-02-14,no,NaN,NaN,NaN
160,NIHZY217YWJA8-task2-2020-02-18,yes,NaN,NaN,NaN


## Process Ratings from Dr. Saloni Sharma

In [7]:
rating_data = read_rating_data()
#Make the 'File_name' consistent with the 'id' in the survey, remove '.webm'
rating_data["File_name"] = rating_data["File_name"].str[:-5]
#Dr. Saloni made some comments and did not give anyrating. Replace those with na
rating_data['Right'] = pd.to_numeric(rating_data['Right'], errors='coerce')
rating_data['Left'] = pd.to_numeric(rating_data['Left'], errors='coerce')
#'Rating' column was not populated. This contains only comments
rating_data = rating_data.drop(columns = ['Rating'])
#Check how many ratings are available
#rating_data[(rating_data["Right"]>=0.0) & (rating_data["Right"]<=4.0)].shape #(186, 3)
#rating_data[(rating_data["Left"]>=0.0) & (rating_data["Left"]<=4.0)].shape #(187,3)

#Only keep the videos where both Left and Right ratings are not N/A
rating_data = rating_data[((~rating_data["Left"].isna()) & (~rating_data["Right"].isna()))]
rating_data

,File_name,Right,Left
0,2017-10-13T17-22-56-936Z44-task2,0.0,1.0
1,2017-10-15T14-47-50-369Z6-task2,1.0,0.0
2,2017-10-15T15-22-00-517Z19-task2,1.0,1.0
4,2017-11-15T22-04-09-657Z62-task2,0.0,0.0
5,2017-11-17T19-41-40-058Z57-task2,0.0,0.0
...,...,...,...
195,NIHYA889LELYV-task2-2019-08-22T13-21-34-659Z-,1.0,0.0
196,NIHYA889LELYV-task2-2020-02-17T14-30-55-008Z-,0.0,0.0
197,NIHYM875FLXFF-task2-2020-03-02T18-39-31-044Z-,1.0,0.0
198,NIHYT60IGVTH5-task2-2019-10-28T14-58-20-600Z-,1.0,0.0


## Read Extracted Features (DeepMAG Frequency Features)

In [8]:
features = read_features()
#dataframe is indexed by 'id'
features_df = pd.DataFrame.from_dict(features, orient='index').rename_axis('id')[["frequency_components"]]
features_df

,frequency_components
id,
2017-08-18T14-59-52-530Z49-task2,"[4795.407543734973, 87.44582005686378, -21.629..."
2017-08-18T15-24-14-004Z53-task2,"[4840.8609913082555, 72.6660925449257, 98.4037..."
2017-08-22T02-01-21-948Z87-task2,"[6229.424923436179, 132.90561812700153, 82.124..."
2017-09-22T18-38-44-872Z33-task2,"[3657.462895487711, -136.14476289189147, -65.1..."
2017-09-28T14-17-07-280Z18-task2,"[6328.233577346078, -835.0499234501573, -264.2..."
...,...
NIHYA889LELYV-task2-2019-08-22T13-21-34-659Z-,"[3099.800019829519, -73.68227785872443, -52.86..."
NIHYA889LELYV-task2-2020-02-17T14-30-55-008Z-,"[3995.0728278472234, -130.80468725127903, -33...."
NIHYM875FLXFF-task2-2020-03-02T18-39-31-044Z-,"[1941.9081372418823, 28.7353869608559, 27.6723..."


## Combine the Dataframes to Build the Final Dataset

In [9]:
def match_superpd_id(row):
    '''
    Match the ID as in SuperPARK survey. 
    For example, NIHYM875FLXFF-task2-2020-03-02T18-39-31-044Z- --> NIHYM875FLXFF-task2-2020-03-02
    '''
    id_str = row['id']
    if 'NIH' in id_str:
        idx = id_str.find('-task2-')
        return id_str[:(idx+17)]
    else:
        return id_str

#All videos that passed the pipeline has features, but may not be rated. Doing a left outer join to combine
rating_data = rating_data.set_index('File_name')
features_and_rating_data = pd.merge(features_df, rating_data, left_index=True, right_index=True, how='left')
features_and_rating_data = features_and_rating_data.reset_index()

#Rename the IDs to match SuperPARK survey IDs before join
features_and_rating_data["id"] = features_and_rating_data.apply(match_superpd_id, axis=1)
#features_and_rating_data

#Inner join concatenated survey and feature dataframes
dataset = pd.merge(features_and_rating_data, combined_survey_data, on="id")
dataset.to_pickle(os.path.join(INPUT_DIR, DATASET_FOR_SEVERITY_RATING))

In [10]:
dataset = dataset[(dataset["Right"]>=0.0) & (dataset["Right"]<=4.0)]
dataset
#dataset[(dataset["Right"]>=0.0) & (dataset["Right"]<=4.0)].shape #(186, 3)
#rating_data[(rating_data["Left"]>=0.0) & (rating_data["Left"]<=4.0)].shape #(187,3)

,id,frequency_components,Right,Left,diagnosed,year-of-diagnosis,last-medication,on_off_state
9,2017-10-13T17-22-56-936Z44-task2,"[7633.094838555125, -107.60068659837572, -106....",0.0,1.0,yes,2013.0,1.500000,NaN
11,2017-10-15T14-47-50-369Z6-task2,"[4821.5990523168675, 166.03696077753133, 112.6...",1.0,0.0,no,NaN,NaN,NaN
12,2017-10-15T15-22-00-517Z19-task2,"[6500.610393670288, -169.95481143522503, 104.6...",1.0,1.0,no,NaN,NaN,NaN
22,2017-11-15T22-04-09-657Z62-task2,"[2715.9280493313804, 148.96789806639282, -106....",0.0,0.0,no,NaN,NaN,NaN
23,2017-11-17T19-41-40-058Z57-task2,"[3212.466625906072, -43.42131950985836, -56.72...",0.0,0.0,no,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
819,NIHXZ891UYEBU-task2-2020-02-05,"[2412.1357409610614, 2.214172041482511, 23.503...",1.0,0.0,no,NaN,NaN,NaN
820,NIHYA889LELYV-task2-2019-08-22,"[3099.800019829519, -73.68227785872443, -52.86...",1.0,0.0,yes,NaN,1.000000,0.0
821,NIHYA889LELYV-task2-2020-02-17,"[3995.0728278472234, -130.80468725127903, -33....",0.0,0.0,yes,NaN,0.833333,0.0
822,NIHYM875FLXFF-task2-2020-03-02,"[1941.9081372418823, 28.7353869608559, 27.6723...",1.0,0.0,no,NaN,NaN,NaN


### Observation
    Total datapoints rated: 179

## Select some files which are not rated, but developed PD for a long time

The following codes were used to send data points to Dr. Sharma for rating. This is not relevant to dataset generation